# Virtual Try-On with Azure OpenAI

This notebook demonstrates how to create a virtual try-on experience for an e-commerce platform using Azure OpenAI's image generation capabilities. Users can upload their own images and "try on" different clothing items and accessories.

## Overview

The process involves:
1. Loading a user's image
2. Loading clothing/accessory items
3. Using Azure OpenAI's image generation to create a realistic composite
4. Displaying the results

This notebook serves as a proof of concept for the full Streamlit web application.

In [ ]:
# Import required libraries
import os
import requests
import base64
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from openai import AzureOpenAI
import uuid
import sys

# Import display_images utility if available in parent directory
sys.path.append('../image-generation-using-aoai')
try:
    from utils import display_images
except ImportError:
    # Fallback display function if the utility isn't available
    def display_images(images, width=None):
        """Display one or multiple images"""
        if not isinstance(images, list):
            images = [images]
            
        for img in images:
            if isinstance(img, str):
                if os.path.exists(img):
                    display(Image.open(img))
                elif img.startswith('http'):
                    response = requests.get(img, stream=True)
                    response.raise_for_status()
                    display(Image.open(BytesIO(response.content)))
                else:
                    # Try as base64
                    try:
                        img_data = base64.b64decode(img)
                        display(Image.open(BytesIO(img_data)))
                    except Exception as e:
                        print(f"Could not process image: {e}")
            elif isinstance(img, Image.Image):
                display(img)
            else:
                print(f"Unsupported image format: {type(img)}")

## Load Azure OpenAI Configuration

We'll load the Azure OpenAI configuration from environment variables or a config file.

In [ ]:
# Try to load from .env file first
load_dotenv()

# Function to load configuration from config.json or environment variables
def load_config():
    try:
        import json
        with open("config.json", "r") as config_file:
            return json.load(config_file)
    except FileNotFoundError:
        # Use environment variables as fallback
        return {
            "imagegen_aoai_resource": os.getenv("imagegen_aoai_resource", ""),
            "imagegen_aoai_endpoint": os.getenv("imagegen_aoai_endpoint", ""),
            "imagegen_aoai_deployment": os.getenv("imagegen_aoai_deployment", ""),
            "imagegen_aoai_api_key": os.getenv("imagegen_aoai_api_key", "")
        }

# Load the configuration
config = load_config()

# Check if configuration is valid
required_keys = ["imagegen_aoai_resource", "imagegen_aoai_endpoint", "imagegen_aoai_deployment", "imagegen_aoai_api_key"]
missing_keys = [key for key in required_keys if not config.get(key)]

if missing_keys:
    print(f"⚠️ Missing required configuration: {', '.join(missing_keys)}")
    print("Please set these values in config.json or as environment variables.")
else:
    print("✅ Configuration loaded successfully")

# Initialize Azure OpenAI client
def get_aoai_client():
    return AzureOpenAI(
        azure_endpoint=config["imagegen_aoai_endpoint"],
        api_version="2025-04-01-preview",
        api_key=config["imagegen_aoai_api_key"]
    )

# Create the client instance
try:
    aoai_client = get_aoai_client()
    print("✅ Azure OpenAI client initialized")
except Exception as e:
    print(f"❌ Error initializing Azure OpenAI client: {str(e)}")

## Helper Functions for Virtual Try-On

Let's define the core functions we'll need for the virtual try-on process.

In [ ]:
def save_base64_image(b64_data, output_path):
    """Save a base64 encoded image to a file"""
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Decode and save image
        image_data = base64.b64decode(b64_data)
        image = Image.open(BytesIO(image_data))
        image.save(output_path)
        return True
    except Exception as e:
        print(f"Error saving image: {str(e)}")
        return False

def load_image_as_base64(image_path):
    """Load an image file and convert it to base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
def generate_try_on_image(user_image_path, item_images, prompt_addon=""):
    """Generate a try-on image using Azure OpenAI API
    
    Args:
        user_image_path (str): Path to the user's image
        item_images (list): List of paths to clothing/accessory items
        prompt_addon (str, optional): Additional prompt text to guide the generation
        
    Returns:
        tuple: (image_path, b64_image) - Path to saved image and base64 image data
    """
    # Prepare URL for image edits
    url = f"https://{config['imagegen_aoai_resource']}.openai.azure.com/openai/deployments/{config['imagegen_aoai_deployment']}/images/edits?api-version=2025-04-01-preview"
    
    # Prepare headers with API key
    headers = {
        "api-key": config["imagegen_aoai_api_key"]
    }
    
    # Prepare the files
    files = [(f"image[]", open(user_image_path, "rb"))]
    for item_path in item_images:
        files.append((f"image[]", open(item_path, "rb")))
    
    # Base prompt for try-on
    base_prompt = """
    Generate a high-quality, photorealistic image of the first person wearing the clothing/accessories shown in the other reference images. 
    Maintain the exact facial features, skin tone, hairstyle, and body type of the first person. 
    Only change their outfit to match the provided catalog items while keeping their identity intact.
    The image should look natural and realistic, with appropriate lighting and background.
    """
    
    if prompt_addon:
        base_prompt += f"\n{prompt_addon}"
    
    data = {
        "prompt": base_prompt,
        "n": 1,
        "size": "1024x1536",  # Portrait orientation for clothing
        "quality": "high",
    }
    
    print("Generating virtual try-on image...")
    response = requests.post(url, headers=headers, files=files, data=data)
    response.raise_for_status()
    
    # Get the image data
    result = response.json()
    b64_image = result["data"][0]["b64_json"]
    
    # Save the generated image
    image_filename = f"generated_{str(uuid.uuid4())[:8]}.png"
    os.makedirs("generated_images", exist_ok=True)
    image_path = os.path.join("generated_images", image_filename)
    
    # Decode and save the image
    save_base64_image(b64_image, image_path)
    print(f"Image saved to: {image_path}")
    
    return image_path, b64_image

## Example 1: Basic Virtual Try-On

Let's try a simple example using the Azure OpenAI image generation API to create a virtual try-on image.

For this demonstration we'll need:
1. A reference image of a person
2. An image of a clothing item or accessory

In [ ]:
# Paths to our images
# Replace with your actual image paths
user_image_path = "../image-generation-using-aoai/images/priya_1.png"  # Your reference person image
item_image_path = "../image-generation-using-aoai/images/woman-shirt.png"  # A clothing item

# Display the input images
print("User Image:")
display_images(user_image_path, width=400)

print("\nClothing Item:")
display_images(item_image_path, width=400)

In [ ]:
# Generate a virtual try-on image
try:
    result_path, b64_image = generate_try_on_image(
        user_image_path=user_image_path,
        item_images=[item_image_path],
        prompt_addon="Make sure the shirt fits perfectly and looks natural on the person."
    )
    
    print("\nGenerated Try-On Image:")
    display_images(result_path, width=500)
except Exception as e:
    print(f"Error generating try-on image: {str(e)}")

## Example 2: Multiple Items Try-On

Now, let's try a more complex example with multiple items (e.g., a shirt and accessories).

In [ ]:
# Paths to our images
# Replace with your actual image paths
user_image_path = "../image-generation-using-aoai/images/priya_1.png"  # Your reference person image
items = [
    "../image-generation-using-aoai/images/woman-shirt.png",  # A shirt
    "../image-generation-using-aoai/images/woman-shoes.png",  # Shoes
    "../image-generation-using-aoai/images/woman-jeans.png"   # Jeans
]

# Display the input images
print("User Image:")
display_images(user_image_path, width=400)

print("\nClothing Items:")
display_images(items, width=200)

In [ ]:
# Generate a virtual try-on image with multiple items
try:
    result_path, b64_image = generate_try_on_image(
        user_image_path=user_image_path,
        item_images=items,
        prompt_addon="Create a complete outfit with the shirt, jeans, and shoes. Make sure all items fit naturally and the overall look is cohesive."
    )
    
    print("\nGenerated Complete Outfit:")
    display_images(result_path, width=500)
except Exception as e:
    print(f"Error generating try-on image: {str(e)}")

## Example 3: Custom Prompt Engineering

Let's experiment with prompt engineering to get better results.

In [ ]:
# Try a more detailed prompt for better results
custom_prompt = """
Generate a professional, high-quality fashion catalog image showing the person wearing the provided clothing items.
The result must be:
- Photorealistic and professional quality
- Maintain exact likeness of the person's face and body proportions
- Show the clothing items fitting naturally and looking high-end
- Use neutral background with soft lighting like a professional fashion photoshoot
- No distortions, unrealistic proportions, or alterations to the person's identity

The final image should look like it belongs in a high-end fashion catalog or e-commerce website.
"""

try:
    result_path, b64_image = generate_try_on_image(
        user_image_path=user_image_path,
        item_images=items,
        prompt_addon=custom_prompt
    )
    
    print("\nGenerated Professional Catalog Image:")
    display_images(result_path, width=500)
except Exception as e:
    print(f"Error generating try-on image: {str(e)}")

## Integration with a Full Web Application

This notebook demonstrates the core functionality of virtual try-on using Azure OpenAI's image generation capabilities. For a full e-commerce implementation, we've created a Streamlit web application that provides a user-friendly interface for:

1. Uploading user photos
2. Browsing a catalog of clothing items and accessories
3. Selecting items to try on
4. Generating and viewing try-on results
5. Downloading and sharing generated images

The application is available in the `virtual-try-on` directory and can be launched with `streamlit run app.py`.

## Conclusion

Virtual try-on using Azure OpenAI provides a powerful way for e-commerce platforms to enhance the shopping experience. Users can see how clothing and accessories would look on them before making a purchase, potentially increasing conversion rates and reducing returns.